# Dependencies

In [ ]:
%pip install torch torchvision
%pip install opencv-python
%pip install numpy
%pip install pillow
%pip install matplotlib

# imports

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

In [ ]:
# -----------------------------
# Augmentation and Transforms
# -----------------------------
class Augmentations:
    @staticmethod
    def get_train_transforms():
        return T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5]),
        ])

    @staticmethod
    def get_test_transforms():
        return T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5]),
        ])

# -----------------------------
# Lightweight Preprocessing: Grayscale Only
# -----------------------------
def lightweight_preprocess(img, target_size=(50, 50)):
    """
    Converts BGR image to grayscale and resizes.
    Returns image as shape (1, H, W) with dtype uint8.
    """
    if img is None:
        return None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, target_size)
    gray = np.expand_dims(gray, axis=0)  # (1, H, W)
    return gray

# -----------------------------
# Dataset
# -----------------------------
class RPSDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.class_map = {'rock': 0, 'paper': 1, 'scissors': 2}
        self.transform = transform
        skipped = 0

        for label in self.class_map.keys():
            class_folder = os.path.join(root_dir, label)
            if not os.path.exists(class_folder):
                continue
            for filename in os.listdir(class_folder):
                if filename.lower().endswith((".jpg", ".png")):
                    path = os.path.join(class_folder, filename)
                    img = cv2.imread(path)
                    proc_img = lightweight_preprocess(img)
                    if proc_img is not None:
                        self.samples.append((path, self.class_map[label]))
                    else:
                        skipped += 1
        if skipped > 0:
            print(f"Skipped {skipped} images due to failed loading/preprocessing")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path)
        proc_img = lightweight_preprocess(img)
        if proc_img is None:
            proc_img = np.zeros((1, 50, 50), dtype=np.uint8)
        # To PIL Image for transform (expects HWC), so squeeze channel
        img_pil = Image.fromarray(proc_img[0])
        if self.transform:
            img_tensor = self.transform(img_pil)
        else:
            img_tensor = torch.tensor(proc_img, dtype=torch.float32) / 255.0
        return img_tensor, label

# -----------------------------
# Model
# -----------------------------
class SimpleCNN(nn.Module):
    def __init__(self, input_size=50, num_classes=3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),  # in_channels=1 for grayscale
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        conv_output_size = (input_size // 4) * (input_size // 4) * 16
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(conv_output_size, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# -----------------------------
# Training Logic
# -----------------------------
class Trainer:
    def __init__(self, model, train_loader, test_loader=None, device=None, lr=0.001):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device or torch.device(
            "cuda" if hasattr(torch, "cuda") and torch.mps.is_available()
            else "cuda" if torch.cuda.is_available()
            else "cpu"
        )
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.model.to(self.device)

    def train(self, num_epochs=10):
        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0
            for imgs, labels in self.train_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(imgs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(self.train_loader)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
            if self.test_loader:
                self.evaluate()

    def evaluate(self):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in self.test_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                outputs = self.model(imgs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Test Accuracy: {accuracy:.2f}%")

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)
        print(f"Model saved at {path}")

# -----------------------------
# Main entry
# -----------------------------
def main():
    train_dir = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/train"
    test_dir = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/test"

    # Datasets
    train_dataset = RPSDataset(
        root_dir=train_dir,
        transform=Augmentations.get_train_transforms()
    )
    test_dataset = RPSDataset(
        root_dir=test_dir,
        transform=Augmentations.get_test_transforms()
    )

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Model
    model = SimpleCNN(input_size=50, num_classes=3)

    # Trainer
    trainer = Trainer(model, train_loader, test_loader)
    trainer.train(num_epochs=10)
    trainer.save_model("model_p_n.pth")

if __name__ == "__main__":
    main()

Epoch 1/10, Loss: 0.7296
Test Accuracy: 50.63%
Epoch 2/10, Loss: 0.6571
Test Accuracy: 64.71%
Epoch 3/10, Loss: 0.6126
Test Accuracy: 67.44%
Epoch 4/10, Loss: 0.5562
Test Accuracy: 67.44%
Epoch 5/10, Loss: 0.5230
Test Accuracy: 73.11%
Epoch 6/10, Loss: 0.4645
Test Accuracy: 73.11%
Epoch 7/10, Loss: 0.4294
Test Accuracy: 78.15%
Epoch 8/10, Loss: 0.3897
Test Accuracy: 80.46%
Epoch 9/10, Loss: 0.3490
Test Accuracy: 80.04%
Epoch 10/10, Loss: 0.3192
Test Accuracy: 80.25%
Model saved at model_p_n.pth


# tests

## "optimizations"

In [ ]:
# -----------------------------
# Augmentation and Transforms "Optimizations"
# -----------------------------
class Augmentations:
    @staticmethod
    def get_transforms():
        return T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5]),
            ])

# -----------------------------
# Improved Preprocessing Pipeline
# -----------------------------
def skin_mask_ycrcb(img):
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    lower = np.array([0, 133, 77], dtype=np.uint8)
    upper = np.array([255, 173, 127], dtype=np.uint8)
    mask = cv2.inRange(ycrcb, lower, upper)
    return mask

def preprocess_image(img, bg_subtractor=None, size=(64, 64)):
    if img is None:
        return None

    # Step 1: Skin detection
    skin = skin_mask_ycrcb(img)

    # Step 2: Background subtraction
    if bg_subtractor:
        fg_mask = bg_subtractor.apply(img)
        combined = cv2.bitwise_and(skin, fg_mask)
    else:
        combined = skin

    # Step 3: Smooth and threshold
    blurred = cv2.GaussianBlur(combined, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)

    # Step 4: Resize and convert to grayscale
    resized = cv2.resize(thresh, size)
    final = np.expand_dims(resized, axis=0)  # shape: (1, H, W)
    return final

# -----------------------------
# Dataset
# -----------------------------
class RPSDataset(Dataset):
    def __init__(self, root_dir, transform=None, bg_subtractor=None):
        self.samples = []
        self.class_map = {'rock': 0, 'paper': 1, 'scissors': 2}
        self.transform = transform
        self.bg_subtractor = bg_subtractor

        for label in self.class_map:
            folder = os.path.join(root_dir, label)
            if not os.path.exists(folder):
                continue
            for fname in os.listdir(folder):
                if fname.lower().endswith(('.jpg', '.png')):
                    path = os.path.join(folder, fname)
                    self.samples.append((path, self.class_map[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = cv2.imread(path)
        proc = preprocess_image(img, self.bg_subtractor)
        if proc is None:
            proc = np.zeros((1, 64, 64), dtype=np.uint8)
        img_pil = Image.fromarray(proc[0])
        if self.transform:
            img_tensor = self.transform(img_pil)
        else:
            img_tensor = torch.tensor(proc, dtype=torch.float32) / 255.0
        return img_tensor, label

# -----------------------------
# MobileNet-style Depthwise CNN
# -----------------------------
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size=3, padding=1, groups=in_ch)
        self.pointwise = nn.Conv2d(in_ch, out_ch, kernel_size=1)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.depthwise(x)
        x = self.activation(x)
        x = self.pointwise(x)
        x = self.activation(x)
        return x

class SimpleMobileNet(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.features = nn.Sequential(
                DepthwiseSeparableConv(1, 8),
                nn.MaxPool2d(2),
                DepthwiseSeparableConv(8, 16),
                nn.MaxPool2d(2),
                )
        self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(16 * 16 * 16, 32),
                nn.ReLU(),
                nn.Linear(32, num_classes)
                )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# -----------------------------
# Training and Evaluation
# -----------------------------
class Trainer:
    def __init__(self, model, train_loader, test_loader=None, device=None, lr=0.001):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

    def train(self, epochs=10):
        for epoch in range(epochs):
            self.model.train()
            running_loss = 0
            for imgs, labels in self.train_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(imgs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
            print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(self.train_loader):.4f}")
            self.evaluate()

    def evaluate(self):
        if not self.test_loader:
            return
        self.model.eval()
        correct = total = 0
        with torch.no_grad():
            for imgs, labels in self.test_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                preds = self.model(imgs).argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        acc = 100 * correct / total
        print(f"Test Accuracy: {acc:.2f}%")

    def save_model(self, path="model_mobile.pth", torchscript_path="model_mobile.pt"):
        torch.save(self.model.state_dict(), path)
        print(f"Model saved: {path}")
        example = torch.rand(1, 1, 64, 64).to(self.device)
        scripted = torch.jit.trace(self.model, example)
        scripted.save(torchscript_path)
        print(f"TorchScript saved: {torchscript_path}")

# -----------------------------
# Main
# -----------------------------
def main():
    train_path = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/train"
    test_path = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/test"

    bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)

    train_set = RPSDataset(train_path, transform=Augmentations.get_transforms(), bg_subtractor=bg_subtractor)
    test_set = RPSDataset(test_path, transform=Augmentations.get_transforms(), bg_subtractor=bg_subtractor)

    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32)

    model = SimpleMobileNet(num_classes=3)
    trainer = Trainer(model, train_loader, test_loader)
    trainer.train(epochs=10)
    trainer.save_model()

if __name__ == "__main__":
    main()

Epoch 1/10, Loss: 0.7367
Test Accuracy: 50.63%
Epoch 2/10, Loss: 0.6939
Test Accuracy: 50.63%
Epoch 3/10, Loss: 0.6734
Test Accuracy: 61.55%
Epoch 4/10, Loss: 0.6460
Test Accuracy: 57.56%
Epoch 5/10, Loss: 0.6336
Test Accuracy: 63.03%
Epoch 6/10, Loss: 0.6094
Test Accuracy: 68.70%
Epoch 7/10, Loss: 0.5915
Test Accuracy: 67.65%
Epoch 8/10, Loss: 0.6084
Test Accuracy: 62.18%
Epoch 9/10, Loss: 0.5884
Test Accuracy: 66.81%
Epoch 10/10, Loss: 0.5784
Test Accuracy: 67.86%
Model saved: model_mobile.pth
TorchScript saved: model_mobile.pt


## no preproc + indiano

In [ ]:
# -----------------------------
# Augmentation and Transforms "no preproc + indiano"
# -----------------------------
class Augmentations:
    @staticmethod
    def get_train_transforms():
        return T.Compose([
            T.RandomResizedCrop(50, scale=(0.8, 1.2), ratio=(0.9, 1.1)),
            T.RandomRotation(30),
            T.ColorJitter(brightness=0.3, contrast=0.3),
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5]),
        ])

    @staticmethod
    def get_test_transforms():
        return T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.5], std=[0.5]),
        ])

# -----------------------------
# Dataset
# -----------------------------
class RPSDataset(Dataset):
    def __init__(self, root_dir, transform=None, class_map=None):
        self.samples = []
        self.transform = transform
        # You can modify this mapping for your specific classes
        self.class_map = class_map or {'rock': 0, 'paper': 1, 'scissors': 2}
        skipped = 0

        for label in self.class_map.keys():
            class_folder = os.path.join(root_dir, label)
            if not os.path.exists(class_folder):
                continue
            for filename in os.listdir(class_folder):
                if filename.lower().endswith((".jpg", ".png")):
                    path = os.path.join(class_folder, filename)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        self.samples.append((path, self.class_map[label]))
                    else:
                        skipped += 1
        if skipped > 0:
            print(f"Skipped {skipped} images due to failed loading")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            img = np.zeros((50, 50), dtype=np.uint8)
        img = cv2.resize(img, (50, 50))
        img_pil = Image.fromarray(img)
        if self.transform:
            img_tensor = self.transform(img_pil)
        else:
            img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0) / 255.0
        return img_tensor, label

# -----------------------------
# Model
# -----------------------------
class LightweightCNN(nn.Module):
    def __init__(self, image_x=50, image_y=50, num_of_classes=3):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=5, stride=5, padding=2)

        # Dynamically determine flatten dimension
        dummy = torch.zeros(1, 1, image_x, image_y)
        x = self.pool1(F.relu(self.conv1(dummy)))
        x = self.pool2(F.relu(self.conv2(x)))
        flat_dim = x.view(1, -1).shape[1]

        self.fc1 = nn.Linear(flat_dim, 1024)
        self.dropout = nn.Dropout(0.6)
        self.fc2 = nn.Linear(1024, num_of_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# -----------------------------
# Training Logic
# -----------------------------
class Trainer:
    def __init__(self, model, train_loader, test_loader=None, device=None, lr=0.001):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device or torch.device(
            "cuda" if torch.cuda.is_available()
            else "cpu"
        )
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.model.to(self.device)

    def train(self, num_epochs=10):
        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0.0
            for imgs, labels in self.train_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(imgs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(self.train_loader)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
            if self.test_loader:
                self.evaluate()

    def evaluate(self):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in self.test_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                outputs = self.model(imgs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Test Accuracy: {accuracy:.2f}%")

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)
        print(f"Model saved at {path}")

# -----------------------------
# Main entry
# -----------------------------
def main():
    train_dir = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/train"
    test_dir = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/test"
    image_x, image_y = 50, 50
    num_classes = 3
    class_map = {'rock': 0, 'paper': 1, 'scissors': 2}

    # Datasets
    train_dataset = RPSDataset(
        root_dir=train_dir,
        transform=Augmentations.get_train_transforms(),
        class_map=class_map
    )
    test_dataset = RPSDataset(
        root_dir=test_dir,
        transform=Augmentations.get_test_transforms(),
        class_map=class_map
    )

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Model
    model = LightweightCNN(image_x=image_x, image_y=image_y, num_of_classes=num_classes)

    # Trainer
    trainer = Trainer(model, train_loader, test_loader)
    trainer.train(num_epochs=10)
    trainer.save_model("model_np_i.pth")

if __name__ == "__main__":
    main()

Epoch 1/10, Loss: 0.7458
Test Accuracy: 56.93%
Epoch 2/10, Loss: 0.6864
Test Accuracy: 58.82%
Epoch 3/10, Loss: 0.6715
Test Accuracy: 65.76%
Epoch 4/10, Loss: 0.6586
Test Accuracy: 65.55%
Epoch 5/10, Loss: 0.6532
Test Accuracy: 67.02%
Epoch 6/10, Loss: 0.6317
Test Accuracy: 68.91%
Epoch 7/10, Loss: 0.6296
Test Accuracy: 72.06%
Epoch 8/10, Loss: 0.6064
Test Accuracy: 64.92%
Epoch 9/10, Loss: 0.6029
Test Accuracy: 72.06%
Epoch 10/10, Loss: 0.5738
Test Accuracy: 73.53%
Model saved at model_np_i.pth


## no preproc + normal

In [ ]:
# -----------------------------
# Augmentation and Transforms "no preproc + normal"
# -----------------------------
class Augmentations:
    @staticmethod
    def get_train_transforms():
        # For raw color images (3 channels)
        return T.Compose([
            T.RandomResizedCrop(50, scale=(0.8, 1.2), ratio=(0.9, 1.1)),
            T.RandomRotation(30),
            T.ColorJitter(brightness=0.3, contrast=0.3),
            T.ToTensor(),
            T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])

    @staticmethod
    def get_test_transforms():
        return T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])

# -----------------------------
# Dataset
# -----------------------------
class RPSDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.class_map = {'rock': 0, 'paper': 1, 'scissors': 2}
        self.transform = transform
        skipped = 0

        for label in self.class_map.keys():
            class_folder = os.path.join(root_dir, label)
            if not os.path.exists(class_folder):
                continue
            for filename in os.listdir(class_folder):
                if filename.lower().endswith((".jpg", ".png")):
                    path = os.path.join(class_folder, filename)
                    img = cv2.imread(path)
                    if img is not None:
                        self.samples.append((path, self.class_map[label]))
                    else:
                        skipped += 1
        if skipped > 0:
            print(f"Skipped {skipped} images due to failed loading")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = cv2.imread(img_path)
        if img is None:
            # fallback to blank image with 3 channels
            img = np.zeros((50, 50, 3), dtype=np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (50, 50))
        img_pil = Image.fromarray(img)
        if self.transform:
            img_tensor = self.transform(img_pil)
        else:
            img_tensor = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32) / 255.0
        return img_tensor, label

# -----------------------------
# Model
# -----------------------------
class SimpleCNN(nn.Module):
    def __init__(self, input_size=50, num_classes=3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # in_channels=3 for RGB
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        conv_output_size = (input_size // 4) * (input_size // 4) * 32
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(conv_output_size, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# -----------------------------
# Training Logic
# -----------------------------
class Trainer:
    def __init__(self, model, train_loader, test_loader=None, device=None, lr=0.001):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device or torch.device(
            "cuda" if hasattr(torch, "cuda") and torch.mps.is_available()
            else "cuda" if torch.cuda.is_available()
            else "cpu"
        )
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.model.to(self.device)

    def train(self, num_epochs=10):
        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0
            for imgs, labels in self.train_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(imgs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            avg_loss = total_loss / len(self.train_loader)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
            if self.test_loader:
                self.evaluate()

    def evaluate(self):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in self.test_loader:
                imgs, labels = imgs.to(self.device), labels.to(self.device)
                outputs = self.model(imgs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Test Accuracy: {accuracy:.2f}%")

    def save_model(self, path):
        torch.save(self.model.state_dict(), path)
        print(f"Model saved at {path}")

# -----------------------------
# Main entry
# -----------------------------
def main():
    train_dir = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/train"
    test_dir = "/content/drive/MyDrive/sassocartaforbici/dataset_solo_persone/test"

    # Datasets
    train_dataset = RPSDataset(
        root_dir=train_dir,
        transform=Augmentations.get_train_transforms()
    )
    test_dataset = RPSDataset(
        root_dir=test_dir,
        transform=Augmentations.get_test_transforms()
    )

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Model
    model = SimpleCNN(input_size=50, num_classes=3)

    # Trainer
    trainer = Trainer(model, train_loader, test_loader)
    trainer.train(num_epochs=10)
    trainer.save_model("model_np_n.pth")

if __name__ == "__main__":
    main()

Epoch 1/10, Loss: 0.7161
Test Accuracy: 59.03%
Epoch 2/10, Loss: 0.6774
Test Accuracy: 63.66%
Epoch 3/10, Loss: 0.6577
Test Accuracy: 65.76%
Epoch 4/10, Loss: 0.6488
Test Accuracy: 60.29%
Epoch 5/10, Loss: 0.6292
Test Accuracy: 70.17%
Epoch 6/10, Loss: 0.6057
Test Accuracy: 72.06%
Epoch 7/10, Loss: 0.5762
Test Accuracy: 73.53%
Epoch 8/10, Loss: 0.5519
Test Accuracy: 76.47%
Epoch 9/10, Loss: 0.5732
Test Accuracy: 71.22%
Epoch 10/10, Loss: 0.5149
Test Accuracy: 75.21%
Model saved at model_np_n.pth
